<a href="https://colab.research.google.com/github/ecsquare/Covid-19/blob/master/RandomforstwithSparke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import random
import numpy as np
from pyspark.sql import Row
from sklearn import neighbors
from pyspark.ml.feature import VectorAssembler


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [28]:
cd /content/gdrive/My\ Drive/Colab\ Notebooks/

/content/gdrive/My Drive/Colab Notebooks


In [30]:
path = '/content/gdrive/My Drive/Colab Notebooks/fow.json'
df = spark.read.json(path)

In [ ]:
df.printSchema()

In [31]:
df.createOrReplaceTempView("df")

In [88]:
from pyspark.sql.functions import lit
df_features = df.select("user.id",
                    "favorite_count",
                     "retweet_count",
                     "user.default_profile",
                     "user.description",
                     "user.favourites_count",
                     "user.followers_count",
                     "user.friends_count",
                     "user.geo_enabled",
                     "user.has_extended_profile",
                     "user.listed_count",
                     "user.location",
                     "user.protected",
                     "user.statuses_count",
                     "user.verified")
df_final = df_features.withColumn("target", lit(0))

In [89]:
df_final.printSchema()

root
 |-- id: long (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- default_profile: boolean (nullable = true)
 |-- description: string (nullable = true)
 |-- favourites_count: long (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- geo_enabled: boolean (nullable = true)
 |-- has_extended_profile: boolean (nullable = true)
 |-- listed_count: long (nullable = true)
 |-- location: string (nullable = true)
 |-- protected: boolean (nullable = true)
 |-- statuses_count: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- target: integer (nullable = false)



MapPartitionsRDD[40] at javaToPython at NativeMethodAccessorImpl.java:0

In [143]:
from pyspark.sql.functions import translate, col
from pyspark.sql import functions as F
from pyspark.sql.types import *

update_func = (F.when(F.col('description') == "", 0)
                .otherwise(1))

df_f2 = df_final.withColumn('description', update_func) 

In [144]:
df_f2.show()

+------------------+--------------+-------------+---------------+-----------+----------------+---------------+-------------+-----------+--------------------+------------+------------+---------+--------------+--------+------+
|                id|favorite_count|retweet_count|default_profile|description|favourites_count|followers_count|friends_count|geo_enabled|has_extended_profile|listed_count|    location|protected|statuses_count|verified|target|
+------------------+--------------+-------------+---------------+-----------+----------------+---------------+-------------+-----------+--------------------+------------+------------+---------+--------------+--------+------+
|         512700138|          1283|          260|          false|          1|              35|        2645847|           47|      false|               false|        5821|            |    false|          4020|    true|     0|
|720293443260456960|             0|           41|          false|          1|            7794|      

In [145]:
update_loc = (F.when(F.col('location') == "", 0)
                .otherwise(1))

df_f = df_f2.withColumn('location', update_loc) 

In [146]:
df_f.show()

+------------------+--------------+-------------+---------------+-----------+----------------+---------------+-------------+-----------+--------------------+------------+--------+---------+--------------+--------+------+
|                id|favorite_count|retweet_count|default_profile|description|favourites_count|followers_count|friends_count|geo_enabled|has_extended_profile|listed_count|location|protected|statuses_count|verified|target|
+------------------+--------------+-------------+---------------+-----------+----------------+---------------+-------------+-----------+--------------------+------------+--------+---------+--------------+--------+------+
|         512700138|          1283|          260|          false|          1|              35|        2645847|           47|      false|               false|        5821|       0|    false|          4020|    true|     0|
|720293443260456960|             0|           41|          false|          1|            7794|         478389|      

In [149]:
splits = df_f.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [156]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
vectorAssembler = VectorAssembler(inputCols=["favorite_count",
                     "retweet_count",
                     "default_profile",
                     "description",
                     "favourites_count",
                     "followers_count",
                     "friends_count",
                     "geo_enabled",
                     "has_extended_profile",
                     "listed_count",
                     "location",
                     "protected",
                     "statuses_count",],
                    outputCol="features")

normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)

In [157]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCol="target", outputCol="targetOHE")

In [160]:
def classification_metrics(prediction):
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    mcEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("HOURLYWindDirectionBucketized")
    accuracy = mcEval.evaluate(prediction)
    print("Accuracy on test data = %g" % accuracy)

In [ ]:
#RF1
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="target", numTrees=30)


pipeline = Pipeline(stages=[vectorAssembler,normalizer])
model = pipeline.fit(df_train)
prediction = model.transform(df_test)
classification_metrics(prediction)